比较 Qwen 和 QwQ 两种 chat_template 在 DPO 训练时是否有差异
 - 只在准备数据上有差异: qwq 的数据要带 /think 标签
 - 应该是一般不调整 system 字段
 - 有一个疑问: 对 /think 的数据进行 DPO 是否会导致模型不愿意输出 /think 标签? -> 应该不会，后面训练的时候可以留意一下

In [42]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("/root/model/qwen")
# tokenizer = AutoTokenizer.from_pretrained("/root/model/qwen-distill")
tokenizer = AutoTokenizer.from_pretrained("/root/model/qwq")


# model = AutoModelForCausalLM.from_pretrained("/root/model")

In [23]:
from datasets import load_from_disk
dataset = load_from_disk("/root/dataset")
dataset

Dataset({
    features: ['rejected', 'rejected_score', 'chosen_score', 'chosen'],
    num_rows: 10000
})

In [36]:
data = dataset[0]
data

{'rejected': [{'content': 'Part 1. Definition\nYou are given a math word problem and you are supposed to apply division mathematical operator on the numbers embedded in the text to answer the following question and then only report the final numerical answer.\nPart 2. Example\nContext: Alex wants to split a collection of eggs into 4 groups. Alex has 8 eggs and 6 marbles. \n\n Question: How many eggs will each group have?\nAnswer: 2\nExplanation: Total eggs each group will get is 8/4 = 2\nPart 3. Exercise\nContext: 6 packs of dvds can be bought with 120 dollars. \n\n Question: How much does each pack cost?\nAnswer:',
   'role': 'user'},
  {'content': 'Your task is to define the mathematical operation to be performed on the given numbers to solve the problem and provide a formula to calculate the answer. In this case, the task involves dividing the total amount of eggs by the number of groups, and your answer will be the number of eggs each group will have. I will provide you with the fo

In [43]:
prompt = ""
chosen = tokenizer.apply_chat_template(data["chosen"], tokenize=False)
rejected = tokenizer.apply_chat_template(data["rejected"], tokenize=False)
# if is_dpo:
#     prompt = tokenizer.apply_chat_template(data[chosen_key][:-1], tokenize=False, add_generation_prompt=True)
#     chosen = chosen[len(prompt) :]
#     rejected = rejected[len(prompt) :]

In [ ]:
# R1-distill, 官方不建议加 system 字段
chosen, rejected

('<｜begin▁of▁sentence｜><｜User｜>Part 1. Definition\nYou are given a math word problem and you are supposed to apply division mathematical operator on the numbers embedded in the text to answer the following question and then only report the final numerical answer.\nPart 2. Example\nContext: Alex wants to split a collection of eggs into 4 groups. Alex has 8 eggs and 6 marbles. \n\n Question: How many eggs will each group have?\nAnswer: 2\nExplanation: Total eggs each group will get is 8/4 = 2\nPart 3. Exercise\nContext: 6 packs of dvds can be bought with 120 dollars. \n\n Question: How much does each pack cost?\nAnswer:<｜Assistant｜>6 packs x $120/pack = $720\nExplanation: To find how much each pack costs, we need to divide the total cost by the number of packs. 120 dollars can buy 6 packs, so each pack costs $120/6 = $20.<｜end▁of▁sentence｜>',
 '<｜begin▁of▁sentence｜><｜User｜>Part 1. Definition\nYou are given a math word problem and you are supposed to apply division mathematical operator o

In [30]:
# qwen
chosen, rejected

('<|im_start|>system\nPlease reason step by step, and put your final answer within \\boxed{}.<|im_end|>\n<|im_start|>user\nPart 1. Definition\nYou are given a math word problem and you are supposed to apply division mathematical operator on the numbers embedded in the text to answer the following question and then only report the final numerical answer.\nPart 2. Example\nContext: Alex wants to split a collection of eggs into 4 groups. Alex has 8 eggs and 6 marbles. \n\n Question: How many eggs will each group have?\nAnswer: 2\nExplanation: Total eggs each group will get is 8/4 = 2\nPart 3. Exercise\nContext: 6 packs of dvds can be bought with 120 dollars. \n\n Question: How much does each pack cost?\nAnswer:<|im_end|>\n<|im_start|>assistant\n6 packs x $120/pack = $720\nExplanation: To find how much each pack costs, we need to divide the total cost by the number of packs. 120 dollars can buy 6 packs, so each pack costs $120/6 = $20.<|im_end|>\n',
 '<|im_start|>system\nPlease reason step

In [44]:
# qwq
chosen, rejected

('<|im_start|>user\nPart 1. Definition\nYou are given a math word problem and you are supposed to apply division mathematical operator on the numbers embedded in the text to answer the following question and then only report the final numerical answer.\nPart 2. Example\nContext: Alex wants to split a collection of eggs into 4 groups. Alex has 8 eggs and 6 marbles. \n\n Question: How many eggs will each group have?\nAnswer: 2\nExplanation: Total eggs each group will get is 8/4 = 2\nPart 3. Exercise\nContext: 6 packs of dvds can be bought with 120 dollars. \n\n Question: How much does each pack cost?\nAnswer:<|im_end|>\n<|im_start|>assistant\n6 packs x $120/pack = $720\nExplanation: To find how much each pack costs, we need to divide the total cost by the number of packs. 120 dollars can buy 6 packs, so each pack costs $120/6 = $20.<|im_end|>\n',
 '<|im_start|>user\nPart 1. Definition\nYou are given a math word problem and you are supposed to apply division mathematical operator on the n

In [ ]:
# qen-distill
prompt = tokenizer.apply_chat_template(data["chosen"][:-1], tokenize=False, add_generation_prompt=True)
chosen = chosen[len(prompt) :]
rejected = rejected[len(prompt) :]
prompt, chosen, rejected

('<｜begin▁of▁sentence｜><｜User｜>Part 1. Definition\nYou are given a math word problem and you are supposed to apply division mathematical operator on the numbers embedded in the text to answer the following question and then only report the final numerical answer.\nPart 2. Example\nContext: Alex wants to split a collection of eggs into 4 groups. Alex has 8 eggs and 6 marbles. \n\n Question: How many eggs will each group have?\nAnswer: 2\nExplanation: Total eggs each group will get is 8/4 = 2\nPart 3. Exercise\nContext: 6 packs of dvds can be bought with 120 dollars. \n\n Question: How much does each pack cost?\nAnswer:<｜Assistant｜><think>\n',
 'x $120/pack = $720\nExplanation: To find how much each pack costs, we need to divide the total cost by the number of packs. 120 dollars can buy 6 packs, so each pack costs $120/6 = $20.<｜end▁of▁sentence｜>',
 'k is to define the mathematical operation to be performed on the given numbers to solve the problem and provide a formula to calculate the 

In [45]:
# qwq
prompt = tokenizer.apply_chat_template(data["chosen"][:-1], tokenize=False, add_generation_prompt=True)
chosen = chosen[len(prompt) :]
rejected = rejected[len(prompt) :]
prompt, chosen, rejected

('<|im_start|>user\nPart 1. Definition\nYou are given a math word problem and you are supposed to apply division mathematical operator on the numbers embedded in the text to answer the following question and then only report the final numerical answer.\nPart 2. Example\nContext: Alex wants to split a collection of eggs into 4 groups. Alex has 8 eggs and 6 marbles. \n\n Question: How many eggs will each group have?\nAnswer: 2\nExplanation: Total eggs each group will get is 8/4 = 2\nPart 3. Exercise\nContext: 6 packs of dvds can be bought with 120 dollars. \n\n Question: How much does each pack cost?\nAnswer:<|im_end|>\n<|im_start|>assistant\n<think>\n',
 'x $120/pack = $720\nExplanation: To find how much each pack costs, we need to divide the total cost by the number of packs. 120 dollars can buy 6 packs, so each pack costs $120/6 = $20.<|im_end|>\n',
 'k is to define the mathematical operation to be performed on the given numbers to solve the problem and provide a formula to calculate 

In [31]:
# qwen
prompt = tokenizer.apply_chat_template(data["chosen"][:-1], tokenize=False, add_generation_prompt=True)
chosen = chosen[len(prompt) :]
rejected = rejected[len(prompt) :]
prompt, chosen, rejected

('<|im_start|>system\nPlease reason step by step, and put your final answer within \\boxed{}.<|im_end|>\n<|im_start|>user\nPart 1. Definition\nYou are given a math word problem and you are supposed to apply division mathematical operator on the numbers embedded in the text to answer the following question and then only report the final numerical answer.\nPart 2. Example\nContext: Alex wants to split a collection of eggs into 4 groups. Alex has 8 eggs and 6 marbles. \n\n Question: How many eggs will each group have?\nAnswer: 2\nExplanation: Total eggs each group will get is 8/4 = 2\nPart 3. Exercise\nContext: 6 packs of dvds can be bought with 120 dollars. \n\n Question: How much does each pack cost?\nAnswer:<|im_end|>\n<|im_start|>assistant\n',
 '6 packs x $120/pack = $720\nExplanation: To find how much each pack costs, we need to divide the total cost by the number of packs. 120 dollars can buy 6 packs, so each pack costs $120/6 = $20.<|im_end|>\n',
 'Your task is to define the mathem